In [1]:
#导入需要的包
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn import datasets
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn import tree, metrics, ensemble
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
import csv
import seaborn as sns
from statsmodels.gam.api import GLMGam, BSplines
from pygam import LogisticGAM
from pygam import LinearGAM, s, f, l
import itertools
from tqdm import tnrange, tqdm_notebook
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
warnings.filterwarnings('ignore')

In [2]:
#导入数据
data = pd.read_csv("rating.csv")
data.head()
X = data.drop(columns=['X0','X1','X2'])
Y = data['X2']
#对行业加入虚拟变量
data_dummy = pd.get_dummies(data, columns = ['X3'],drop_first = True)
#分出自变量和因变量（这个要等不均衡处理完再来做）
X_dm = data_dummy.drop(columns=['X0','X1','X2'])
Y_dm = data_dummy['X2']
print(X_dm,Y_dm)

            X4      X5       X6           X7       X8      X9      X10  \
0       5.9159  2.4106  61.1055      24.5261  65.3785  2.8884  58.5661   
1      36.1012  0.0000        0      17.8494   0.0000  0.0000   0.0000   
2      44.3443  0.1627  12.8816      48.6845  72.9207  3.6929  99.9027   
3      16.5825  0.1688  12.2076       3.1198  56.1185  2.2789  76.4917   
4      37.1069  0.0000        0      58.8424   0.0000  0.0000   0.0000   
...        ...     ...      ...          ...      ...     ...      ...   
1787  -53.0100  0.0000        0      16.7343   0.0000  0.0000   0.0000   
1788   -6.1506  0.0000        0       3.3864   0.0000  0.0000   0.0000   
1789   -1.4506  0.0000        0      -9.4893   0.0000  0.0000   0.0000   
1790   -1.2954  0.0000        0  -3,782.9384   0.0000  0.0000   0.0000   
1791   -1.0637  0.0000        0       9.0371   0.0000  0.0000   0.0000   

         X11     X12     X13  ... X3_批发和零售业  X3_教育  X3_文化、体育和娱乐业  \
0     0.7429  0.4541  0.5668  ...         0

In [3]:
#数据类型转换，为了可以处理缺失值
X_con_list=['X4','X5','X6','X7','X8','X9','X10','X11','X12','X13','X14','X15']
data_new =data
for i in X_con_list:
    data_new[i] = pd.to_numeric(data[i],errors='coerce')
#缺失值处理（全部替换为均值）-还没好
X3_list=['制造业', '信息传输、软件和信息技术服务业', '建筑业', '水利、环境和公共设施管理业', '租赁和商务服务业', '房地产业', '电力、热力、燃气及水生产和供应业', '批发和零售业', '采矿业', '交通运输、仓储和邮政业', '金融业', '卫生和社会工作', '农、林、牧、渔业', '科学研究和技术服务业', '文化、体育和娱乐业', '综合', '教育', '住宿和餐饮业']
X3_list_pinyin=['zhizao','xinxi','jianzhu','shuili','zulin','fangdi','dianli','pifa','caikuang','jiaotong','jinrong','weisheng','kexue','nonglin','wenhua','zonghe','jiaoyu','zhusu']
X_con_list=['X4','X5','X6','X7','X8','X9','X10','X11','X12','X13','X14','X15']


zhizao = data_new[data_new['X3'] == '制造业']
zhizao['X4'].replace(0,np.mean(zhizao['X4']),inplace=True)
zhizao['X5'].replace(0,np.mean(zhizao['X5']),inplace=True)
zhizao['X6'].replace(0,np.mean(zhizao['X6']),inplace=True)
zhizao['X7'].replace(0,np.mean(zhizao['X7']),inplace=True)
zhizao['X8'].replace(0,np.mean(zhizao['X8']),inplace=True)
zhizao['X9'].replace(0,np.mean(zhizao['X9']),inplace=True)
zhizao['X10'].replace(0,np.mean(zhizao['X10']),inplace=True)
zhizao['X11'].replace(0,np.mean(zhizao['X11']),inplace=True)
zhizao['X12'].replace(0,np.mean(zhizao['X12']),inplace=True)
zhizao['X13'].replace(0,np.mean(zhizao['X13']),inplace=True)
zhizao['X14'].replace(0,np.mean(zhizao['X14']),inplace=True)
zhizao['X15'].replace(0,np.mean(zhizao['X15']),inplace=True)

xinxi = data_new[data_new['X3'] == '信息传输、软件和信息技术服务业']
xinxi['X4'].replace(0,np.mean(xinxi['X4']),inplace=True)
xinxi['X5'].replace(0,np.mean(xinxi['X5']),inplace=True)
xinxi['X6'].replace(0,np.mean(xinxi['X6']),inplace=True)
xinxi['X7'].replace(0,np.mean(xinxi['X7']),inplace=True)
xinxi['X8'].replace(0,np.mean(xinxi['X8']),inplace=True)
xinxi['X9'].replace(0,np.mean(xinxi['X9']),inplace=True)
xinxi['X10'].replace(0,np.mean(xinxi['X10']),inplace=True)
xinxi['X11'].replace(0,np.mean(xinxi['X11']),inplace=True)
xinxi['X12'].replace(0,np.mean(xinxi['X12']),inplace=True)
xinxi['X13'].replace(0,np.mean(xinxi['X13']),inplace=True)
xinxi['X14'].replace(0,np.mean(xinxi['X14']),inplace=True)
xinxi['X15'].replace(0,np.mean(xinxi['X15']),inplace=True)

jianzhu = data_new[data_new['X3'] == '建筑业']
jianzhu['X4'].replace(0,np.mean(jianzhu['X4']),inplace=True)
jianzhu['X5'].replace(0,np.mean(jianzhu['X5']),inplace=True)
jianzhu['X6'].replace(0,np.mean(jianzhu['X6']),inplace=True)
jianzhu['X7'].replace(0,np.mean(jianzhu['X7']),inplace=True)
jianzhu['X8'].replace(0,np.mean(jianzhu['X8']),inplace=True)
jianzhu['X9'].replace(0,np.mean(jianzhu['X9']),inplace=True)
jianzhu['X10'].replace(0,np.mean(jianzhu['X10']),inplace=True)
jianzhu['X11'].replace(0,np.mean(jianzhu['X11']),inplace=True)
jianzhu['X12'].replace(0,np.mean(jianzhu['X12']),inplace=True)
jianzhu['X13'].replace(0,np.mean(jianzhu['X13']),inplace=True)
jianzhu['X14'].replace(0,np.mean(jianzhu['X14']),inplace=True)
jianzhu['X15'].replace(0,np.mean(jianzhu['X15']),inplace=True)

shuili = data_new[data_new['X3'] == '水利、环境和公共设施管理业']
shuili['X4'].replace(0,np.mean(shuili['X4']),inplace=True)
shuili['X5'].replace(0,np.mean(shuili['X5']),inplace=True)
shuili['X6'].replace(0,np.mean(shuili['X6']),inplace=True)
shuili['X7'].replace(0,np.mean(shuili['X7']),inplace=True)
shuili['X8'].replace(0,np.mean(shuili['X8']),inplace=True)
shuili['X9'].replace(0,np.mean(shuili['X9']),inplace=True)
shuili['X10'].replace(0,np.mean(shuili['X10']),inplace=True)
shuili['X11'].replace(0,np.mean(shuili['X11']),inplace=True)
shuili['X12'].replace(0,np.mean(shuili['X12']),inplace=True)
shuili['X13'].replace(0,np.mean(shuili['X13']),inplace=True)
shuili['X14'].replace(0,np.mean(shuili['X14']),inplace=True)
shuili['X15'].replace(0,np.mean(shuili['X15']),inplace=True)

zulin = data_new[data_new['X3'] == '租赁和商务服务业']
zulin['X4'].replace(0,np.mean(zulin['X4']),inplace=True)
zulin['X5'].replace(0,np.mean(zulin['X5']),inplace=True)
zulin['X6'].replace(0,np.mean(zulin['X6']),inplace=True)
zulin['X7'].replace(0,np.mean(zulin['X7']),inplace=True)
zulin['X8'].replace(0,np.mean(zulin['X8']),inplace=True)
zulin['X9'].replace(0,np.mean(zulin['X9']),inplace=True)
zulin['X10'].replace(0,np.mean(zulin['X10']),inplace=True)
zulin['X11'].replace(0,np.mean(zulin['X11']),inplace=True)
zulin['X12'].replace(0,np.mean(zulin['X12']),inplace=True)
zulin['X13'].replace(0,np.mean(zulin['X13']),inplace=True)
zulin['X14'].replace(0,np.mean(zulin['X14']),inplace=True)
zulin['X15'].replace(0,np.mean(zulin['X15']),inplace=True)

fangdi = data_new[data_new['X3'] == '房地产业']
fangdi['X4'].replace(0,np.mean(fangdi['X4']),inplace=True)
fangdi['X5'].replace(0,np.mean(fangdi['X5']),inplace=True)
fangdi['X6'].replace(0,np.mean(fangdi['X6']),inplace=True)
fangdi['X7'].replace(0,np.mean(fangdi['X7']),inplace=True)
fangdi['X8'].replace(0,np.mean(fangdi['X8']),inplace=True)
fangdi['X9'].replace(0,np.mean(fangdi['X9']),inplace=True)
fangdi['X10'].replace(0,np.mean(fangdi['X10']),inplace=True)
fangdi['X11'].replace(0,np.mean(fangdi['X11']),inplace=True)
fangdi['X12'].replace(0,np.mean(fangdi['X12']),inplace=True)
fangdi['X13'].replace(0,np.mean(fangdi['X13']),inplace=True)
fangdi['X14'].replace(0,np.mean(fangdi['X14']),inplace=True)
fangdi['X15'].replace(0,np.mean(fangdi['X15']),inplace=True)

dianli = data_new[data_new['X3'] == '电力、热力、燃气及水生产和供应业']
dianli['X4'].replace(0,np.mean(dianli['X4']),inplace=True)
dianli['X5'].replace(0,np.mean(dianli['X5']),inplace=True)
dianli['X6'].replace(0,np.mean(dianli['X6']),inplace=True)
dianli['X7'].replace(0,np.mean(dianli['X7']),inplace=True)
dianli['X8'].replace(0,np.mean(dianli['X8']),inplace=True)
dianli['X9'].replace(0,np.mean(dianli['X9']),inplace=True)
dianli['X10'].replace(0,np.mean(dianli['X10']),inplace=True)
dianli['X11'].replace(0,np.mean(dianli['X11']),inplace=True)
dianli['X12'].replace(0,np.mean(dianli['X12']),inplace=True)
dianli['X13'].replace(0,np.mean(dianli['X13']),inplace=True)
dianli['X14'].replace(0,np.mean(dianli['X14']),inplace=True)
dianli['X15'].replace(0,np.mean(dianli['X15']),inplace=True)

pifa = data_new[data_new['X3'] == '批发和零售业']
pifa['X4'].replace(0,np.mean(pifa['X4']),inplace=True)
pifa['X5'].replace(0,np.mean(pifa['X5']),inplace=True)
pifa['X6'].replace(0,np.mean(pifa['X6']),inplace=True)
pifa['X7'].replace(0,np.mean(pifa['X7']),inplace=True)
pifa['X8'].replace(0,np.mean(pifa['X8']),inplace=True)
pifa['X9'].replace(0,np.mean(pifa['X9']),inplace=True)
pifa['X10'].replace(0,np.mean(pifa['X10']),inplace=True)
pifa['X11'].replace(0,np.mean(pifa['X11']),inplace=True)
pifa['X12'].replace(0,np.mean(pifa['X12']),inplace=True)
pifa['X13'].replace(0,np.mean(pifa['X13']),inplace=True)
pifa['X14'].replace(0,np.mean(pifa['X14']),inplace=True)
pifa['X15'].replace(0,np.mean(pifa['X15']),inplace=True)

caikuang = data_new[data_new['X3'] == '采矿业']
caikuang['X4'].replace(0,np.mean(caikuang['X4']),inplace=True)
caikuang['X5'].replace(0,np.mean(caikuang['X5']),inplace=True)
caikuang['X6'].replace(0,np.mean(caikuang['X6']),inplace=True)
caikuang['X7'].replace(0,np.mean(caikuang['X7']),inplace=True)
caikuang['X8'].replace(0,np.mean(caikuang['X8']),inplace=True)
caikuang['X9'].replace(0,np.mean(caikuang['X9']),inplace=True)
caikuang['X10'].replace(0,np.mean(caikuang['X10']),inplace=True)
caikuang['X11'].replace(0,np.mean(caikuang['X11']),inplace=True)
caikuang['X12'].replace(0,np.mean(caikuang['X12']),inplace=True)
caikuang['X13'].replace(0,np.mean(caikuang['X13']),inplace=True)
caikuang['X14'].replace(0,np.mean(caikuang['X14']),inplace=True)
caikuang['X15'].replace(0,np.mean(caikuang['X15']),inplace=True)

jiaotong = data_new[data_new['X3'] == '交通运输、仓储和邮政业']
jiaotong['X4'].replace(0,np.mean(jiaotong['X4']),inplace=True)
jiaotong['X5'].replace(0,np.mean(jiaotong['X5']),inplace=True)
jiaotong['X6'].replace(0,np.mean(jiaotong['X6']),inplace=True)
jiaotong['X7'].replace(0,np.mean(jiaotong['X7']),inplace=True)
jiaotong['X8'].replace(0,np.mean(jiaotong['X8']),inplace=True)
jiaotong['X9'].replace(0,np.mean(jiaotong['X9']),inplace=True)
jiaotong['X10'].replace(0,np.mean(jiaotong['X10']),inplace=True)
jiaotong['X11'].replace(0,np.mean(jiaotong['X11']),inplace=True)
jiaotong['X12'].replace(0,np.mean(jiaotong['X12']),inplace=True)
jiaotong['X13'].replace(0,np.mean(jiaotong['X13']),inplace=True)
jiaotong['X14'].replace(0,np.mean(jiaotong['X14']),inplace=True)
jiaotong['X15'].replace(0,np.mean(jiaotong['X15']),inplace=True)

jinrong = data_new[data_new['X3'] == '金融业']
jinrong['X4'].replace(0,np.mean(jinrong['X4']),inplace=True)
jinrong['X5'].replace(0,np.mean(jinrong['X5']),inplace=True)
jinrong['X6'].replace(0,np.mean(jinrong['X6']),inplace=True)
jinrong['X7'].replace(0,np.mean(jinrong['X7']),inplace=True)
jinrong['X8'].replace(0,np.mean(jinrong['X8']),inplace=True)
jinrong['X9'].replace(0,np.mean(jinrong['X9']),inplace=True)
jinrong['X10'].replace(0,np.mean(jinrong['X10']),inplace=True)
jinrong['X11'].replace(0,np.mean(jinrong['X11']),inplace=True)
jinrong['X12'].replace(0,np.mean(jinrong['X12']),inplace=True)
jinrong['X13'].replace(0,np.mean(jinrong['X13']),inplace=True)
jinrong['X14'].replace(0,np.mean(jinrong['X14']),inplace=True)
jinrong['X15'].replace(0,np.mean(jinrong['X15']),inplace=True)

weisheng = data_new[data_new['X3'] == '卫生和社会工作']
weisheng['X4'].replace(0,np.mean(weisheng['X4']),inplace=True)
weisheng['X5'].replace(0,np.mean(weisheng['X5']),inplace=True)
weisheng['X6'].replace(0,np.mean(weisheng['X6']),inplace=True)
weisheng['X7'].replace(0,np.mean(weisheng['X7']),inplace=True)
weisheng['X8'].replace(0,np.mean(weisheng['X8']),inplace=True)
weisheng['X9'].replace(0,np.mean(weisheng['X9']),inplace=True)
weisheng['X10'].replace(0,np.mean(weisheng['X10']),inplace=True)
weisheng['X11'].replace(0,np.mean(weisheng['X11']),inplace=True)
weisheng['X12'].replace(0,np.mean(weisheng['X12']),inplace=True)
weisheng['X13'].replace(0,np.mean(weisheng['X13']),inplace=True)
weisheng['X14'].replace(0,np.mean(weisheng['X14']),inplace=True)
weisheng['X15'].replace(0,np.mean(weisheng['X15']),inplace=True)

nonglin = data_new[data_new['X3'] == '农、林、牧、渔业']
nonglin['X4'].replace(0,np.mean(nonglin['X4']),inplace=True)
nonglin['X5'].replace(0,np.mean(nonglin['X5']),inplace=True)
nonglin['X6'].replace(0,np.mean(nonglin['X6']),inplace=True)
nonglin['X7'].replace(0,np.mean(nonglin['X7']),inplace=True)
nonglin['X8'].replace(0,np.mean(nonglin['X8']),inplace=True)
nonglin['X9'].replace(0,np.mean(nonglin['X9']),inplace=True)
nonglin['X10'].replace(0,np.mean(nonglin['X10']),inplace=True)
nonglin['X11'].replace(0,np.mean(nonglin['X11']),inplace=True)
nonglin['X12'].replace(0,np.mean(nonglin['X12']),inplace=True)
nonglin['X13'].replace(0,np.mean(nonglin['X13']),inplace=True)
nonglin['X14'].replace(0,np.mean(nonglin['X14']),inplace=True)
nonglin['X15'].replace(0,np.mean(nonglin['X15']),inplace=True)

kexue = data_new[data_new['X3'] == '科学研究和技术服务业']
kexue['X4'].replace(0,np.mean(kexue['X4']),inplace=True)
kexue['X5'].replace(0,np.mean(kexue['X5']),inplace=True)
kexue['X6'].replace(0,np.mean(kexue['X6']),inplace=True)
kexue['X7'].replace(0,np.mean(kexue['X7']),inplace=True)
kexue['X8'].replace(0,np.mean(kexue['X8']),inplace=True)
kexue['X9'].replace(0,np.mean(kexue['X9']),inplace=True)
kexue['X10'].replace(0,np.mean(kexue['X10']),inplace=True)
kexue['X11'].replace(0,np.mean(kexue['X11']),inplace=True)
kexue['X12'].replace(0,np.mean(kexue['X12']),inplace=True)
kexue['X13'].replace(0,np.mean(kexue['X13']),inplace=True)
kexue['X14'].replace(0,np.mean(kexue['X14']),inplace=True)
kexue['X15'].replace(0,np.mean(kexue['X15']),inplace=True)

wenhua = data_new[data_new['X3'] == '文化、体育和娱乐业']
wenhua['X4'].replace(0,np.mean(wenhua['X4']),inplace=True)
wenhua['X5'].replace(0,np.mean(wenhua['X5']),inplace=True)
wenhua['X6'].replace(0,np.mean(wenhua['X6']),inplace=True)
wenhua['X7'].replace(0,np.mean(wenhua['X7']),inplace=True)
wenhua['X8'].replace(0,np.mean(wenhua['X8']),inplace=True)
wenhua['X9'].replace(0,np.mean(wenhua['X9']),inplace=True)
wenhua['X10'].replace(0,np.mean(wenhua['X10']),inplace=True)
wenhua['X11'].replace(0,np.mean(wenhua['X11']),inplace=True)
wenhua['X12'].replace(0,np.mean(wenhua['X12']),inplace=True)
wenhua['X13'].replace(0,np.mean(wenhua['X13']),inplace=True)
wenhua['X14'].replace(0,np.mean(wenhua['X14']),inplace=True)
wenhua['X15'].replace(0,np.mean(wenhua['X15']),inplace=True)

zonghe  = data_new[data_new['X3'] == '综合']
zonghe['X4'].replace(0,np.mean(zonghe['X4']),inplace=True)
zonghe['X5'].replace(0,np.mean(zonghe['X5']),inplace=True)
zonghe['X6'].replace(0,np.mean(zonghe['X6']),inplace=True)
zonghe['X7'].replace(0,np.mean(zonghe['X7']),inplace=True)
zonghe['X8'].replace(0,np.mean(zonghe['X8']),inplace=True)
zonghe['X9'].replace(0,np.mean(zonghe['X9']),inplace=True)
zonghe['X10'].replace(0,np.mean(zonghe['X10']),inplace=True)
zonghe['X11'].replace(0,np.mean(zonghe['X11']),inplace=True)
zonghe['X12'].replace(0,np.mean(zonghe['X12']),inplace=True)
zonghe['X13'].replace(0,np.mean(zonghe['X13']),inplace=True)
zonghe['X14'].replace(0,np.mean(zonghe['X14']),inplace=True)
zonghe['X15'].replace(0,np.mean(zonghe['X15']),inplace=True)

jiaoyu = data_new[data_new['X3'] == '教育']
jiaoyu['X4'].replace(0,np.mean(jiaoyu['X4']),inplace=True)
jiaoyu['X5'].replace(0,np.mean(jiaoyu['X5']),inplace=True)
jiaoyu['X6'].replace(0,np.mean(jiaoyu['X6']),inplace=True)
jiaoyu['X7'].replace(0,np.mean(jiaoyu['X7']),inplace=True)
jiaoyu['X8'].replace(0,np.mean(jiaoyu['X8']),inplace=True)
jiaoyu['X9'].replace(0,np.mean(jiaoyu['X9']),inplace=True)
jiaoyu['X10'].replace(0,np.mean(jiaoyu['X10']),inplace=True)
jiaoyu['X11'].replace(0,np.mean(jiaoyu['X11']),inplace=True)
jiaoyu['X12'].replace(0,np.mean(jiaoyu['X12']),inplace=True)
jiaoyu['X13'].replace(0,np.mean(jiaoyu['X13']),inplace=True)
jiaoyu['X14'].replace(0,np.mean(jiaoyu['X14']),inplace=True)
jiaoyu['X15'].replace(0,np.mean(jiaoyu['X15']),inplace=True)

zhusu = data_new[data_new['X3'] == '住宿和餐饮业']
zhusu['X4'].replace(0,np.mean(zhusu['X4']),inplace=True)
zhusu['X5'].replace(0,np.mean(zhusu['X5']),inplace=True)
zhusu['X6'].replace(0,np.mean(zhusu['X6']),inplace=True)
zhusu['X7'].replace(0,np.mean(zhusu['X7']),inplace=True)
zhusu['X8'].replace(0,np.mean(zhusu['X8']),inplace=True)
zhusu['X9'].replace(0,np.mean(zhusu['X9']),inplace=True)
zhusu['X10'].replace(0,np.mean(zhusu['X10']),inplace=True)
zhusu['X11'].replace(0,np.mean(zhusu['X11']),inplace=True)
zhusu['X12'].replace(0,np.mean(zhusu['X12']),inplace=True)
zhusu['X13'].replace(0,np.mean(zhusu['X13']),inplace=True)
zhusu['X14'].replace(0,np.mean(zhusu['X14']),inplace=True)
zhusu['X15'].replace(0,np.mean(zhusu['X15']),inplace=True)
data_new_nomiss=pd.concat([zhizao,xinxi,jianzhu,shuili,zulin,fangdi,dianli,pifa,caikuang,jiaotong,jinrong,weisheng,nonglin,wenhua,zonghe,jiaoyu,zhusu])
data_new_nomiss

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15
0,000422.SZ,湖北宜化,14,制造业,5.9159,2.410600,61.10550,24.5261,65.378500,2.888400,58.566100,0.742900,0.454100,0.566800,31.272600,66.5144
1,000777.SZ,中核科技,14,制造业,36.1012,0.550153,5.55303,17.8494,25.260246,1.136484,39.882881,1.056232,0.437094,0.814092,0.924276,54.5167
3,002102.SZ,ST冠福,14,制造业,16.5825,0.168800,12.20760,3.1198,56.118500,2.278900,76.491700,1.150400,0.255000,0.727300,282.133600,48.9444
4,002177.SZ,*ST御银,14,制造业,37.1069,0.550153,5.55303,58.8424,25.260246,1.136484,39.882881,1.056232,0.437094,0.814092,0.924276,51.2486
7,002374.SZ,中锐股份,14,制造业,-10.1040,0.550153,5.55303,1.4427,25.260246,1.136484,39.882881,1.056232,0.437094,0.814092,0.924276,50.8399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837,002659.SZ,凯文教育,16,教育,-25.1205,-0.054250,-1.69860,-1.7668,17.333950,0.833375,26.932075,2.240800,2.064450,2.171650,-206.233950,52.0241
1007,600730.SH,中国高科,16,教育,376.0265,-0.147000,-4.77760,71.6103,15.997100,1.190400,43.335000,8.433800,8.148100,8.302000,-686.965500,182.2211
755,000428.SZ,华天酒店,16,住宿和餐饮业,50.8881,-0.138167,-0.91010,14.4209,33.866233,1.387300,39.403067,0.669133,0.362067,0.524667,2.414450,61.5150
1013,601007.SH,金陵饭店,16,住宿和餐饮业,104.0412,0.107000,2.66290,29.3709,43.291600,1.763400,82.405600,1.375500,0.624700,0.948900,4.828900,48.9256


In [4]:
X = data_new_nomiss.drop(columns=['X0','X1','X2'])
Y = data_new_nomiss['X2']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
test_set = pd.concat([y_test,X_test],axis=1)
test_set.to_csv('./test_set.csv')

In [5]:
train_set = pd.concat([y_train,X_train],axis=1)
train_set.to_csv('./train_set.csv')

In [5]:
'''#评级数据不均衡检验
from collections import Counter
data = pd.concat([y_train,X_train],axis=1)
y_list = list(data[u'X2'])
y_count = Counter(y_list)
print(y_count)


y_list = list(data[u'X3'])
y_count = Counter(y_list)
print(y_count)
#重采样
rate17 = data[data['X2'] == 17]  # 将多数类别
rate1 = data[data['X2'] == 1]  # 将少数类别的样本放在data0
index = np.random.randint(
    len(rate1), size=int(len(rate17) - len(rate1)))  # 随机给定上采样取出样本的序号
up_rate1 = rate1.iloc[list(index)]  # 上采样

rate17 = data[data['X2'] == 17]  # 将多数类别
rate2 = data[data['X2'] == 2]  # 将少数类别的样本放在data0
index = np.random.randint(
    len(rate2), size=int((len(rate17) - len(rate2))))  # 随机给定上采样取出样本的序号
up_rate2 = rate2.iloc[list(index)]  # 上采样

rate17 = data[data['X2'] == 17]  # 将多数类别
rate3 = data[data['X2'] == 3]  # 将少数类别的样本放在data0
index = np.random.randint(
    len(rate3), size=int((len(rate17) - len(rate3))))  # 随机给定上采样取出样本的序号
up_rate3 = rate3.iloc[list(index)]  # 上采样

rate17 = data[data['X2'] == 17]  # 将多数类别
rate4 = data[data['X2'] == 4]  # 将少数类别的样本放在data0
index = np.random.randint(
    len(rate4), size=int((len(rate17) - len(rate4))))
up_rate4 = rate4.iloc[list(index)]  # 上采样

rate17 = data[data['X2'] == 17]  # 将多数类别
rate5 = data[data['X2'] == 5]  # 将少数类别的样本放在data0
index = np.random.randint(
    len(rate5), size=int((len(rate17) - len(rate5))))
up_rate5 = rate5.iloc[list(index)]  # 上采样

rate17 = data[data['X2'] == 17]  # 将多数类别
rate6 = data[data['X2'] == 6]  # 将少数类别的样本放在data0
index = np.random.randint(
    len(rate6), size=int((len(rate17) - len(rate6))))
up_rate6 = rate6.iloc[list(index)]  # 上采样

rate17 = data[data['X2'] == 17]  # 将多数类别
rate7 = data[data['X2'] == 7]  # 将少数类别的样本放在data0
index = np.random.randint(
    len(rate7), size=int((len(rate17) - len(rate7))))
up_rate7 = rate7.iloc[list(index)]  # 上采样

rate17 = data[data['X2'] == 17]  # 将多数类别
rate8 = data[data['X2'] == 8]  # 将少数类别的样本放在data0
index = np.random.randint(
    len(rate8), size=int((len(rate17) - len(rate8))))
up_rate8 = rate8.iloc[list(index)]  # 上采样

rate17 = data[data['X2'] == 17]  # 将多数类别
rate9 = data[data['X2'] == 9]  # 将少数类别的样本放在data0
index = np.random.randint(
    len(rate9), size=int((len(rate17) - len(rate9))))
up_rate9 = rate9.iloc[list(index)]  # 上采样

rate17 = data[data['X2'] == 17]  # 将多数类别
rate9 = data[data['X2'] == 9]  # 将少数类别的样本放在data0
index = np.random.randint(
    len(rate9), size=int((len(rate17) - len(rate9))))
up_rate9 = rate9.iloc[list(index)]  # 上采样

rate17 = data[data['X2'] == 17]  # 将多数类别
rate10 = data[data['X2'] == 10]  # 将少数类别的样本放在data0
index = np.random.randint(
    len(rate10), size=int((len(rate17))))
up_rate10 = rate10.iloc[list(index)]  # 上采样

rate17 = data[data['X2'] == 17]  # 将多数类别
rate11 = data[data['X2'] == 11]  # 将少数类别的样本放在data0
index = np.random.randint(
    len(rate11), size=int((len(rate17))))
up_rate11 = rate11.iloc[list(index)]  # 上采样

rate17 = data[data['X2'] == 17]  # 将多数类别
rate12 = data[data['X2'] == 12]  # 将少数类别的样本放在data0
index = np.random.randint(
    len(rate12), size=int((len(rate17))))
up_rate12 = rate12.iloc[list(index)]  # 上采样

rate17 = data[data['X2'] == 17]  # 将多数类别
rate13 = data[data['X2'] == 13]  # 将少数类别的样本放在data0
index = np.random.randint(
    len(rate13), size=int((len(rate17))))
up_rate13 = rate13.iloc[list(index)]  # 上采样

rate17 = data[data['X2'] == 17]  # 将多数类别
rate14 = data[data['X2'] == 14]  # 将少数类别的样本放在data0
index = np.random.randint(
    len(rate14), size=int((len(rate17))))
up_rate14 = rate14.iloc[list(index)]  # 上采样

rate17 = data[data['X2'] == 17]  # 将多数类别
rate15 = data[data['X2'] == 15]  # 将少数类别的样本放在data0
index = np.random.randint(
    len(rate15), size=int((len(rate17))))
up_rate15 = rate15.iloc[list(index)]  # 上采样

rate17 = data[data['X2'] == 17]  # 将多数类别
rate16 = data[data['X2'] == 16]  # 将少数类别的样本放在data0
index = np.random.randint(
    len(rate16), size=int((len(rate17))))
up_rate16 = rate16.iloc[list(index)]  # 上采样

rate17 = data[data['X2'] == 17]  # 将多数类别
rate18 = data[data['X2'] == 18]  # 将少数类别的样本放在data0
index = np.random.randint(
    len(rate18), size=int((len(rate17))))
up_rate18 = rate18.iloc[list(index)]  # 上采样

rate17 = data[data['X2'] == 17]  # 将多数类别
rate19 = data[data['X2'] == 19]  # 将少数类别的样本放在data0
index = np.random.randint(
    len(rate19), size=int((len(rate17))))
up_rate19 = rate19.iloc[list(index)]  # 上采样

data_new = pd.concat(
    [up_rate1, up_rate2, up_rate3, up_rate4, up_rate5, up_rate6, up_rate7, up_rate8, up_rate9, up_rate10, up_rate11,
     up_rate12, up_rate13, up_rate14, up_rate15, up_rate16, rate17, up_rate18, up_rate19])
data_new.to_csv('./train.csv')
#处理后数据均衡检验
from collections import Counter

y_list = list(data_new[u'X2'])
y_count = Counter(y_list)
print(y_count)'''

Counter({17: 406, 19: 276, 16: 266, 18: 209, 15: 141, 14: 31, 1: 24, 13: 14, 10: 10, 12: 10, 11: 7, 9: 6, 3: 5, 8: 5, 2: 4, 6: 3, 4: 2, 5: 2, 7: 1})
Counter({'制造业': 809, '金融业': 93, '房地产业': 71, '信息传输、软件和信息技术服务业': 68, '电力、热力、燃气及水生产和供应业': 67, '交通运输、仓储和邮政业': 64, '建筑业': 60, '批发和零售业': 52, '采矿业': 43, '水利、环境和公共设施管理业': 34, '租赁和商务服务业': 19, '文化、体育和娱乐业': 14, '农、林、牧、渔业': 14, '教育': 4, '综合': 4, '住宿和餐饮业': 3, '卫生和社会工作': 3})
Counter({10: 406, 11: 406, 12: 406, 13: 406, 14: 406, 15: 406, 16: 406, 17: 406, 18: 406, 19: 406, 7: 405, 4: 404, 5: 404, 6: 403, 2: 402, 3: 401, 8: 401, 9: 400, 1: 382})
